# Исследование надёжности заёмщиков

## Описание

Определить влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок на данных о платёжеспособности клиентов.


## Цель

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Вывод

Наличие 1-2 детей увеличивает процент невозврата кредита в срок на 2% (до 10%).

Официально зарегистривавшие брак или бывшие в браке имеют просрочки в 6-8% случаев, в то время как одинокие заемщики до 10%.

Наибольший риск невозврата кредита у заемщиков с доходом от 100 000 до 200 000 рублей (до 9%).

Выше всего риски просрочек при покупке автомобиля или кредита на образование - более 9%.

## Шаг 1. Изучим данные

In [1]:
# импортируем библиотеку
import pandas as pd

In [2]:
# прочитаем данные
df = pd.read_csv('data.csv')

In [3]:
# посмотрим данные
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


children — количество детей в семье

days_employed — общий трудовой стаж в днях

dob_years — возраст клиента в годах

education — уровень образования клиента

education_id — идентификатор уровня образования

family_status — семейное положение

family_status_id — идентификатор семейного положения

gender — пол клиента

income_type — тип занятости

debt — имел ли задолженность по возврату кредитов

total_income — ежемесячный доход

purpose — цель получения кредита

In [6]:
# посмотрим типы данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [7]:
# посмотрим процент пропущенных значений
df.isna().sum()*100/len(df)

children             0.000000
days_employed       10.099884
dob_years            0.000000
education            0.000000
education_id         0.000000
family_status        0.000000
family_status_id     0.000000
gender               0.000000
income_type          0.000000
debt                 0.000000
total_income        10.099884
purpose              0.000000
dtype: float64

In [8]:
# поменяем формат данных, чтобы данные были в обычном формате, без е
pd.options.display.float_format ='{: .3f}'.format 

In [9]:
# посмотрим статистические данные
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000,19351.000,21525.000,21525.000,21525.000,21525.000,19351.000
mean,0.539,63046.498,43.293,0.817,0.973,0.081,167422.302
std,1.382,140827.312,12.575,0.548,1.420,0.273,102971.566
min,-1.000,-18388.950,0.000,0.000,0.000,0.000,20667.264
25%,0.000,-2747.424,33.000,1.000,0.000,0.000,103053.153
50%,0.000,-1203.370,42.000,1.000,0.000,0.000,145017.938
75%,1.000,-291.096,53.000,1.000,1.000,0.000,203435.068
max,20.000,401755.400,75.000,4.000,4.000,1.000,2265604.029


**Вывод**

В данных имеются NaN значения в total_income (ежемесячный доход) и в days_employed (трудовой стаж) в одинаковом количестве. Доля таких пропущенных значений 10%.
Посмотрим сколько таких значений и есть ли в них схожести. Так же в данных о детях есть число -1 и 20 детей, что нестандартно. И 0 лет в возрасте. И значения float стоит перевести в целочисленные значения.

## Шаг 2. Предобработка данных

### Обработка пропусков

Найдем пропуски в данных о ежемесячном доходе методом isna()

In [10]:
df[df['total_income'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


In [11]:
# сравним распределение значения для среза с пропусками
df['total_income'].value_counts(normalize=True)

253875.639    0.000
157691.851    0.000
70113.903     0.000
116196.519    0.000
157205.835    0.000
              ...  
168880.592    0.000
148042.721    0.000
60039.334     0.000
175979.763    0.000
82047.419     0.000
Name: total_income, Length: 19351, dtype: float64

**Вывод**

Схожих данных в nan нет, таких как, что это только пенсионеры, или только те, кто гасит кредит без просрочки. Значит предполагаем, что они не заполнены случайным образом, по ошибке выгрузке или иных случаев. Примерно 10% пропущенных данных. 

Поскольку запросить новые данные мы не можем, или узнать причину их не заполненности, будем только предполагать.

Можем предположить, что доход зависит от типа занятости. В связи с чем можем заполнить 10% пропущенных значений медианными значениями в зависимости от занятости. Однако указать в выводе на число пропущенных необходимо, так как то влияет на конечный результат.

Данные в столбце days_employed говорят о стаже работе в днях согласно заголовка, но имеют отрицательные значения, завышенные значения,такие как 401755.400 дней стажа.

То есть данные нам предоставлены не совсем некорректные. Ориентироваться на них мы не можем.

На решение поставленных вопросов данные о стаже нам не нужны, потому можем от данного столбца отказаться.

In [12]:
# Посмотрим процент не заполненных данных по типу занятости
df.groupby('income_type')['total_income'].apply(lambda x : x.isna().mean())

income_type
безработный        0.000
в декрете          0.000
госслужащий        0.101
компаньон          0.100
пенсионер          0.107
предприниматель    0.500
сотрудник          0.099
студент            0.000
Name: total_income, dtype: float64

Примерно по 1% данных не заполнено по типу занятости, кроме предпринимателя (5%). Так как нет возможности уточнить сведения у заказчика. Заполним данные о доходах total_income медианными значениями в группировке по типу занятости income_type

In [15]:
df['total_income'] = df['total_income'].fillna((df.groupby('income_type'))['total_income'].transform('median'))

Проверим результат

In [16]:
df[df['total_income'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Пропущенные значения в столбце отсутствуют

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Удалим столбец days_employed (стаж работы) методом dropna() с Nan значениями

In [18]:
df = df.dropna(axis = 'columns')

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   dob_years         21525 non-null  int64  
 2   education         21525 non-null  object 
 3   education_id      21525 non-null  int64  
 4   family_status     21525 non-null  object 
 5   family_status_id  21525 non-null  int64  
 6   gender            21525 non-null  object 
 7   income_type       21525 non-null  object 
 8   debt              21525 non-null  int64  
 9   total_income      21525 non-null  float64
 10  purpose           21525 non-null  object 
dtypes: float64(1), int64(5), object(5)
memory usage: 1.8+ MB


### Замена типа данных

Изменим в total_income (ежемесячный доход) вещественный тип float на целочисленный int с помощью метода astype(), так как он позволяет переводить данные в любой тип, в отличие от to_numeric().

In [20]:
df['total_income'] = df['total_income'].astype('int64')

C:\Users\mdvor\AppData\Local\Temp\ipykernel_12576\3602641882.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['total_income'] = df['total_income'].astype('int64')


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   dob_years         21525 non-null  int64 
 2   education         21525 non-null  object
 3   education_id      21525 non-null  int64 
 4   family_status     21525 non-null  object
 5   family_status_id  21525 non-null  int64 
 6   gender            21525 non-null  object
 7   income_type       21525 non-null  object
 8   debt              21525 non-null  int64 
 9   total_income      21525 non-null  int64 
 10  purpose           21525 non-null  object
dtypes: int64(6), object(5)
memory usage: 1.8+ MB


In [22]:
df.head()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


### Обработка дубликатов

Найдем дубликаты методом duplicated() и посчитаем их количество методом sum().

In [23]:
df.duplicated().sum()

54

54 дублирующиеся строки. Удалим их методом drop_duplicates()

In [24]:
df.drop_duplicates(inplace=True)

C:\Users\mdvor\AppData\Local\Temp\ipykernel_12576\3006716147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


Посмотрим неявные дубликаты в столбцах методом unique().

In [25]:
for col in df:
    print('Столбец:', col) # название столбца
    print(df[col].unique()) # уникальные записи столбца

Столбец: children
[ 1  0  3  2 -1  4 20  5]
Столбец: dob_years
[42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]
Столбец: education
['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
Столбец: education_id
[0 1 2 3 4]
Столбец: family_status
['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
Столбец: family_status_id
[0 1 2 3 4]
Столбец: gender
['F' 'M' 'XNA']
Столбец: income_type
['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']
Столбец: debt
[0 1]
Столбец: total_income
[253875 112080 145885 ...  89672 244093  82047]
Столбец: purpose
['покупка жилья' 'приобретение автомобиля' 'дополнительное образование

**ВЫВОД**

Данные о детях содержат странные значения -1 и 20.

Данные о возрасте имеют 0 возраст.

Данные об образовании и браке стоит перевести в строчное написание.

Данные пола имеют неизвестное значение 'XNA'.

Данные о цели кредита содержат схожие цели, стоит привести их к одному слову.

In [26]:
# посмотрим количество записей с -1 ребенком
df[df['children'] == -1]['children'].count()

47

In [27]:
# посчитаем процент таких записей
df[df['children'] == -1]['children'].count()/len(df)*100

0.21889991150854643

In [28]:
# проверим содержание записей
df[df['children'] == -1]

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
291,-1,46,среднее,1,гражданский брак,1,F,сотрудник,0,102816,профильное образование
705,-1,50,среднее,1,женат / замужем,0,F,госслужащий,0,137882,приобретение автомобиля
742,-1,57,среднее,1,женат / замужем,0,F,сотрудник,0,64268,дополнительное образование
800,-1,54,среднее,1,Не женат / не замужем,4,F,пенсионер,0,86293,дополнительное образование
941,-1,57,Среднее,1,женат / замужем,0,F,пенсионер,0,118514,на покупку своего автомобиля
1363,-1,55,СРЕДНЕЕ,1,женат / замужем,0,F,компаньон,0,69550,профильное образование
1929,-1,38,среднее,1,Не женат / не замужем,4,M,сотрудник,0,109121,покупка жилья
2073,-1,42,среднее,1,в разводе,3,F,компаньон,0,162638,покупка жилья
3814,-1,26,Среднее,1,гражданский брак,1,F,госслужащий,0,131892,на проведение свадьбы
4201,-1,41,среднее,1,женат / замужем,0,F,госслужащий,0,226375,операции со своей недвижимостью


In [29]:
# посмотрим число записей с количеством детей 20
df[df['children'] == 20]['children'].count()

76

In [30]:
# посчитаем процент таких записий
df[df['children'] == 20]['children'].count()/len(df)*100

0.35396581435424523

In [31]:
# посмотрим содержание записей
df[df['children'] == 20]

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,21,среднее,1,женат / замужем,0,M,компаньон,0,145334,покупка жилья
720,20,44,среднее,1,женат / замужем,0,F,компаньон,0,112998,покупка недвижимости
1074,20,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518,получение образования
2510,20,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474,операции с коммерческой недвижимостью
2941,20,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739,на покупку автомобиля
...,...,...,...,...,...,...,...,...,...,...,...
21008,20,40,среднее,1,женат / замужем,0,F,сотрудник,1,133524,свой автомобиль
21325,20,37,среднее,1,женат / замужем,0,F,компаньон,0,102986,профильное образование
21390,20,53,среднее,1,женат / замужем,0,M,компаньон,0,172357,покупка жилой недвижимости
21404,20,52,среднее,1,женат / замужем,0,M,компаньон,0,156629,операции со своей недвижимостью


44 записи с -1 ребенком и 67 записей с 20 детьми. Такое маловероятно. Видимо это ошибка ввода сотрудника. Число таких записей составляет менее 1% всех данных. Такое небольшое число возможно удалить или заменить такие данные предположительно на 1 и 2 ребенка или заполнить медианой, например, по возрасту.

В реальности возникновение таких ошибок лучше уточнить у заказчика или программиста, что они означают, если ответа нет, то такие данные лучше удалить, так как 100% полагаться на них мы не можем.

Предположим, что это мы получили данные, что это ошибка сотрудника: "-1" - это 1 ребенок, а "20" - это 2 ребенка. Заменим в соответствии.

In [32]:
df['children'] = df['children'].replace(-1, 1)
df['children'] = df['children'].replace(20, 2)

C:\Users\mdvor\AppData\Local\Temp\ipykernel_12576\3302947032.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['children'] = df['children'].replace(-1, 1)
C:\Users\mdvor\AppData\Local\Temp\ipykernel_12576\3302947032.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['children'] = df['children'].replace(20, 2)


In [33]:
# проверим результат
df['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

В данных о возрасте присутствует значение 0.

Проверим число таких записей и их распределение по занятости

In [34]:
df[df['dob_years'] == 0]['dob_years'].count()

101

In [35]:
df[df['dob_years'] == 0].groupby('income_type')['dob_years'].count()

income_type
госслужащий     6
компаньон      20
пенсионер      20
сотрудник      55
Name: dob_years, dtype: int64

In [36]:
# проверим долю значений 0 возраста по типу занятости
df.groupby('income_type')['dob_years'].apply(lambda x : x[x==0].mean())

income_type
безработный          NaN
в декрете            NaN
госслужащий        0.000
компаньон          0.000
пенсионер          0.000
предприниматель      NaN
сотрудник          0.000
студент              NaN
Name: dob_years, dtype: float64

101 запись с 0 возрастом. К какому то определенному типу занятости не соответствует 0 значение.
В целом возраст клиента не важен для определения поставленных целей. Поэтому изменять или удалять данные не имеет смысла.

Данные об образовании и браке имеют различное написание. Приведем данные к нижнему регистру.

In [37]:
df['education'] = df['education'].str.lower()

C:\Users\mdvor\AppData\Local\Temp\ipykernel_12576\623256638.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['education'] = df['education'].str.lower()


In [38]:
# проверим результат
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [39]:
# приводим данные о статусе брака к нижнему регистру
df['family_status'] = df['family_status'].str.lower()

C:\Users\mdvor\AppData\Local\Temp\ipykernel_12576\3102737373.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['family_status'] = df['family_status'].str.lower()


In [40]:
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

Тип пола имеет значение XNA кроме женского и мужского.

Посмотрим записи с таким значением.

In [41]:
df[df['gender'] == 'XNA']

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905,покупка недвижимости


Одна запись с типом XNA. Данный тип gender - пол не влияет на цели нашего исследования, можем его сохранить без изменений.

Отсортируем данные о ежемесячном доходе

In [42]:
df['total_income'].sort_values()

14585      20667
13006      21205
16174      21367
1598       21695
14276      21895
          ...   
17178    1711309
20809    1715018
9169     1726276
19606    2200852
12412    2265604
Name: total_income, Length: 21471, dtype: int64

Данные о зп больше 0. Изменять что-либо нет необходимости.

In [43]:
# посмотрим цели кредита
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Необходимо цели получения кредита разбить на категории, предварительно заменив "ремонт жилью" на "ремонт".

Напишем функцию замены схожих слов на определенную цель кредита.
Перезапишем данные в столбце purpose.

In [44]:
df['purpose'] = df['purpose'].replace('ремонт жилью', 'ремонт')

C:\Users\mdvor\AppData\Local\Temp\ipykernel_12576\335468012.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['purpose'] = df['purpose'].replace('ремонт жилью', 'ремонт')


In [45]:
def create_category_purpose(row):
    for i in row['purpose']:
        if 'строител' in row['purpose']:
                return 'строительство'
        elif 'автомобил' in row['purpose']:
            return 'автомобиль'
        elif 'жиль' in row['purpose'] or 'недвижимост' in row['purpose']:
            return 'недвижимость'
        elif 'образован' in row['purpose']:
            return 'образование'
        elif 'ремонт' in row['purpose']:
            return 'ремонт'
        elif 'свадьб' in row['purpose']:
            return 'свадьба'
df['purpose'] = df.apply(create_category_purpose, axis=1)

C:\Users\mdvor\AppData\Local\Temp\ipykernel_12576\4104191713.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['purpose'] = df.apply(create_category_purpose, axis=1)


In [46]:
# проверим данные
df['purpose'].unique()

array(['недвижимость', 'автомобиль', 'образование', 'свадьба',
       'строительство', 'ремонт'], dtype=object)

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21471 entries, 0 to 21524
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21471 non-null  int64 
 1   dob_years         21471 non-null  int64 
 2   education         21471 non-null  object
 3   education_id      21471 non-null  int64 
 4   family_status     21471 non-null  object
 5   family_status_id  21471 non-null  int64 
 6   gender            21471 non-null  object
 7   income_type       21471 non-null  object
 8   debt              21471 non-null  int64 
 9   total_income      21471 non-null  int64 
 10  purpose           21471 non-null  object
dtypes: int64(6), object(5)
memory usage: 2.0+ MB


In [48]:
df.duplicated().sum()

271

Данные отстались в прежнем количестве. Появилось 271 явных дубликатов. Удалим их.

In [49]:
df.drop_duplicates(inplace=True)
# проверим число дубликатов
df.duplicated().sum()

C:\Users\mdvor\AppData\Local\Temp\ipykernel_12576\203501858.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


0

**ВЫВОД**

Работа с данными проведена. Проведем дальнейший анализ.

### Категоризация данных

Посмотрим на данные в столбце о количестве детей

In [50]:
df.groupby('children')['debt'].count()

children
0    13889
1     4814
2     2118
3      329
4       41
5        9
Name: debt, dtype: int64

Имеет смысл объединить 3 и более детей в одну группу, так как их не так много в общем количестве.
Разделим на категории:
0  - нет детей,
1 - один ребенок,
2 - два ребенка,
более 3 - многодетные

In [51]:
def children_group(row):
    if row == 0:
        return '0 детей'
    if row == 1:
        return '1 ребенок'
    if row == 2:
        return '2 ребенка'
    else:
        return 'многодетные'

Создадим новый столбец children_group по числу детей

In [53]:
df['children_group'] = df['children'].apply(children_group)
df.head()

C:\Users\mdvor\AppData\Local\Temp\ipykernel_12576\3919888094.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['children_group'] = df['children'].apply(children_group)


,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,children_group
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,1 ребенок
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,1 ребенок
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость,0 детей
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,многодетные
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,0 детей


Посмотрим минимум и максимум доходов

In [54]:
df['total_income'].describe()

count      21200.000
mean      165636.745
std        98714.727
min        20667.000
25%       106967.250
50%       142594.000
75%       196685.000
max      2265604.000
Name: total_income, dtype: float64

Проведем категорирование уровня доходов на 5 групп, исходя из доходов:

1 - от 0 до 100 000,

2 - от 100 000 до 150 000,

3 - от 150 000 до 200 000,

4 - от 200 000 до 250 000,

5 - от 250 000 и выше.

In [55]:
def total_income_group(row):
    if row > 0 and row < 100000:
        return 'от 0 до 100 000 р'
    if row >= 100000 and row < 150000:
        return 'от 100 000 до 150 000 р'
    if row >= 150000 and row < 200000:
        return 'от 150 000 до 200 000 р'
    if row >= 200000 and row < 250000:
        return 'от 200 000 до 250 000 р'
    else:
        return 'от 250 000 р и выше'

Запишем данные о группах в новый столбец

In [56]:
df['total_income_group'] = df['total_income'].apply(total_income_group)
df.head()

C:\Users\mdvor\AppData\Local\Temp\ipykernel_12576\537585671.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['total_income_group'] = df['total_income'].apply(total_income_group)


,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,children_group,total_income_group
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,1 ребенок,от 250 000 р и выше
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,1 ребенок,от 100 000 до 150 000 р
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость,0 детей,от 100 000 до 150 000 р
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,многодетные,от 250 000 р и выше
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,0 детей,от 150 000 до 200 000 р


In [57]:
# проверим стандарное распределение доходов в группах
df.groupby('total_income_group').agg(число_клиентов=('total_income','count'),std=('total_income','std')).sort_values('total_income_group')

,число_клиентов,std
total_income_group,,
от 0 до 100 000 р,4463,16682.952
от 100 000 до 150 000 р,6940,14541.538
от 150 000 до 200 000 р,4730,13837.423
от 200 000 до 250 000 р,2254,14068.057
от 250 000 р и выше,2813,140994.870


Во 2 группе отклонение велико, но в остальных группах примерно одинакова. Поэтому оставим данное деление на группы.

## Шаг 3. Проверка зависимостей

**- Есть ли зависимость между наличием детей и возвратом кредита в срок?**

Посмотрим соотношение задолженности по группам детей

In [62]:
(df.groupby('children_group')['debt'].mean())*100

children_group
0 детей        7.646
1 ребенок      9.244
2 ребенка      9.537
многодетные    8.179
Name: debt, dtype: float64

In [60]:
# проверим число записей с необходимым количеством детей
(df.groupby('children_group')['debt'].count())

children_group
0 детей        13889
1 ребенок       4814
2 ребенка       2118
многодетные      379
Name: debt, dtype: int64

**Вывод**

При наличии 1-2 детей количество допущенных просрочек больше примерно на 2% по сравнению с теми, у кого нет детей.

То есть можно сказать, что наличие 1-2 детей увеличивает процент невозврата кредита в срок до 2%.

**- Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [63]:
(df.groupby('family_status')['debt'].mean())*100

family_status
в разводе                7.125
вдовец / вдова           6.646
гражданский брак         9.406
женат / замужем          7.659
не женат / не замужем    9.817
Name: debt, dtype: float64

In [64]:
# посмотрим на количество записей
(df.groupby('family_status')['debt'].count())*100

family_status
в разводе                 119300
вдовец / вдова             94800
гражданский брак          412500
женат / замужем          1214300
не женат / не замужем     279100
Name: debt, dtype: int64

**Вывод**

Наибольший процент просроченных платежей у тех, кто не в законном браке - до 10%.

Наименьший процент у вдовцов и вдов - 6,6%.

Чуть выше риск у женатых и разведенных - до 8%.

То есть официально зарегистривавшие брак или бывшие в браке люди ответственнее подходят к погашению кредита.
У неженатых риски до 4% выше.

**- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [65]:
(df.groupby('total_income_group')['debt'].mean())*100

total_income_group
от 0 до 100 000 р          7.932
от 100 000 до 150 000 р    8.977
от 150 000 до 200 000 р    8.562
от 200 000 до 250 000 р    7.276
от 250 000 р и выше        6.897
Name: debt, dtype: float64

**Вывод**

Наименьшее число просрочки у заемщиков, чьи доходы более 200 000 рублей - около 7%.

Примерно на 1% выше у тех кто получает до 100 000 рублей (7-8%)

И набольший риск до 9% у тех, у кого доходы более 100 000 и менее 200 000 рублей.

Таким образом, примерно на 2% больше риска невозврата кредита у людей, чьи доходы от 100 000 до 200 000 рублей.

В этой части возможно рассмотреть более детальные группы по доходам. Такие вопросы предпочтительно согласовывать с заказчиком.

**- Как разные цели кредита влияют на его возврат в срок?**

In [66]:
(df.groupby('purpose')['debt'].mean())*100

purpose
автомобиль       9.410
недвижимость     7.372
образование      9.334
ремонт           5.766
свадьба          8.066
строительство    7.696
Name: debt, dtype: float64

In [67]:
# посмотрим на количество записей
(df.groupby('purpose')['debt'].count())*100

purpose
автомобиль       427200
недвижимость     818000
образование      396400
ремонт            60700
свадьба          230600
строительство    187100
Name: debt, dtype: int64

**Вывод**

Меньше всего просрочек по кредитам, взятым на ремонт - до 6%.

На недвижимость, строительство и свадьбу - до 8%.

Более всего риски при покупке автомобиля или кредита на образование - выше 9%.

## Шаг 4. Общий вывод

1. Имеется зависимость между наличием детей и возвратом кредита в срок: да.

   Наличие 1-2 детей увеличивает процент невозврата кредита в срок на 2%.


2. Имеется зависимость между семейным положением и возвратом кредита в срок: да.
   
   Официально зарегистривавшие брак люди ответственнее подходят к погашению кредита (6-8%), в то время как одинокие заемщики и    не зарегистрированные в официальном браке имеют риски не возврата кредита в срок на 3-4% выше (9-10%).


3. Имеется зависимость между уровнем дохода и возвратом кредита в срок: да.
   
   При доходах до 250000 рублей - примерно на 2% больше риска невозврата кредита у людей, чьи доходы от 100 000 до 200 000    
   рублей.


4. А так же разные цели кредита влияют на его возврат в срок: выше всего риски при покупке автомобиля или кредита на  
   образование - более 9%, на 1% меньше при покупке недвижимости, строительства или на свадьбу - до 8%, самый низкий риск при  
   получении кредита на ремонт - до 6%.